# Format Ciprofloxacin Kill-Curve Data of N. ghonorrhoeae WHO-M Strain

In [5]:
import os

import chi.plots
import pandas as pd

In [6]:
directory = os.getcwd()
name = '/WHOM_CIP.txt'
raw_data = pd.read_csv(directory + name, delimiter="\t")
raw_data

,sample,count,dilution,CFU.per.ml,MIC,antibiotic.conc,hours
0,WHOM_1,2,1000,200000.0,0.001563,0.003125,-4
1,WHOM_2,2,1000,200000.0,0.015625,0.031250,-4
2,WHOM_3,2,1000,200000.0,0.031250,0.062500,-4
3,WHOM_4,2,1000,200000.0,0.062500,0.125000,-4
4,WHOM_5,2,1000,200000.0,0.125000,0.250000,-4
...,...,...,...,...,...,...,...
91,WHOM_8,18,1,1800.0,1.000000,2.000000,6
92,WHOM_9,9,1,900.0,2.000000,4.000000,6
93,WHOM_10,7,1,700.0,4.000000,8.000000,6
94,WHOM_11,6,1,600.0,8.000000,16.000000,6


## Parse data into chi format

In [7]:
# Shift time such that first measurement is at t=0
temp = raw_data.copy()
temp['hours'] += 4

# Create container for the formatted data
data = pd.DataFrame(columns=[
    'ID', 'Time', 'Time unit', 'Biomarker', 'Measurement', 'Biomarker unit', 
    'Concentration', 'Concentration unit'])

# Loop through individuals
for _id ,sample_id in enumerate(temp['sample'].unique()):
    # Mask relevant data for individual
    mask = temp['sample'] == sample_id
    temp2 = temp[mask]

    # Make sure that ID determines concentration exposure
    assert len(temp2['antibiotic.conc'].unique()) == 1

    # Parse data
    container = pd.DataFrame({
        'ID': _id + 1,
        'Time': temp2['hours'],
        'Time unit': 'h',
        'Biomarker': 'Bacterial count',
        'Measurement': temp2['CFU.per.ml'],
        'Biomarker unit': 'CFU/ml'})
    container = container.append(pd.DataFrame({
        'ID': _id + 1,
        'Time': 4,
        'Time unit': 'h',
        'Concentration': temp2['antibiotic.conc'].unique() * 10**3,
        'Concentration unit': 'ng/ml'}))

    # Append data of individual to dataset
    data = data.append(container)

data

,ID,Time,Time unit,Biomarker,Measurement,Biomarker unit,Concentration,Concentration unit
0,1,0,h,Bacterial count,200000.0,CFU/ml,NaN,NaN
12,1,4,h,Bacterial count,8000000.0,CFU/ml,NaN,NaN
24,1,5,h,Bacterial count,10000000.0,CFU/ml,NaN,NaN
36,1,6,h,Bacterial count,11000000.0,CFU/ml,NaN,NaN
48,1,7,h,Bacterial count,32000000.0,CFU/ml,NaN,NaN
...,...,...,...,...,...,...,...,...
59,12,7,h,Bacterial count,7000.0,CFU/ml,NaN,NaN
71,12,8,h,Bacterial count,1100.0,CFU/ml,NaN,NaN
83,12,9,h,Bacterial count,1000.0,CFU/ml,NaN,NaN
95,12,10,h,Bacterial count,600.0,CFU/ml,NaN,NaN


## Visualise data

In [8]:
fig = chi.plots.PDTimeSeriesPlot()
fig.add_data(data)
fig.show()

## Export data

In [9]:
directory = os.path.dirname(os.getcwd())
name = '/test.csv'
data.to_csv(path_or_buf=directory + name, index=False)